# Getting Started

The purpose of the present `Getting Started` section is to give a quick overview of the main objects, methods and functions of the Python `iode` library.
To get a more detailed presentation of all capabilities of `iode`, read the next sections of the tutorial.
    
The [API Reference](../api.rst#api-reference) section of the documentation give you the list of all objects, methods and functions with their individual documentation and examples.

The [Equivalence IODE Report Commands and IODE Python](../equivalence.rst#equivalence-iode-report-commands-and-iode-python) section contains *equivalence tables* between the IODE report syntax and the Python `iode` syntax.

To use the Python `iode` library, the first thing to do is to import objects and functions you need from it:

In [41]:
from iode import (SAMPLE_DATA_DIR, NA, comments, equations, identities, lists, scalars, 
                  tables, variables, EqMethod)

To know the version of the `iode` library installed on your machine, type:

In [42]:
from iode import __version__
__version__

'7.0.0'

To get the list of objects and functions available in the `iode` li library, use the Python function `dir()`:

In [43]:
import iode
dir(iode)

['AdjustmentMethod',
 'Comments',
 'EQUATION_METHODS_LIST',
 'EQ_TEST_NAMES',
 'ESTIMATION_EPS',
 'ESTIMATION_MAXIT',
 'EditAndEstimateEquations',
 'EqMethod',
 'EqTest',
 'Equation',
 'Equations',
 'FileType',
 'HighToLowType',
 'IODE_DATABASE_TYPE_NAMES',
 'IODE_FILE_TYPES',
 'IODE_FILE_TYPE_NAMES',
 'IODE_LANGUAGES_LIST',
 'Identities',
 'Identity',
 'IodeFileType',
 'IodeTypes',
 'Lists',
 'LowToHighMethod',
 'LowToHighType',
 'NA',
 'Path',
 'Period',
 'SAMPLE_DATA_DIR',
 'Sample',
 'Scalar',
 'Scalars',
 'Simulation',
 'SimulationInitialization',
 'SimulationSort',
 'Table',
 'TableCellAlign',
 'TableCellFont',
 'TableCellType',
 'TableGraphAlign',
 'TableGraphAxis',
 'TableGraphGrid',
 'TableGraphType',
 'TableLang',
 'TableLineType',
 'Tables',
 'Variables',
 'VarsMode',
 'WriteFileExt',
 '_IODE_DEFAULT_DATABASE_FILENAME',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '

To print the documentation of an object, method or function in a Python interactive console, use the `help()` function:

In [44]:
# ---- print documentation of a function or method ----
help(equations.load)

Help on method load in module iode.iode_python:

load(filepath: 'str') method of iode.iode_python.Equations instance
    load(self, filepath: str)
    
    Load objects stored in file 'filepath' into the current database.
    Erase the database before to load the file.
    
    Parameters
    ----------
    filepath: str
        path to the file to load
    
    Examples
    --------
    >>> from iode import SAMPLE_DATA_DIR
    >>> from iode import comments, variables
    >>> comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
    >>> len(comments)
    317
    
    >>> variables.load(f"{SAMPLE_DATA_DIR}/fun.var")
    >>> len(variables)
    394



## Load IODE objects

To load IODE objects from a binary file (i.e. with extension `.cmt`, `.eqs`, `.idt`, `.lst`, `.scl`, `.tbl`, `.var`) or from an ASCII file (i.e. with extension `.ac`, `.ae`, `.ai`, `.al`, `.as`, `.at`, `.av`), use the [load()](../_generated/iode.Comments.load.rst#iode.Comments.load) method of the corresponding object. For example:

In [45]:
# ---- load equations, identities, scalars and variables ----
# Note: test binary and ASCII 'fun' files are located in the 'SAMPLE_DATA_DIR' 
#       directory of the 'iode' package
comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
equations.load(f"{SAMPLE_DATA_DIR}/fun.eqs")
identities.load(f"{SAMPLE_DATA_DIR}/fun.idt")
lists.load(f"{SAMPLE_DATA_DIR}/fun.lst")
scalars.load(f"{SAMPLE_DATA_DIR}/fun.scl")
tables.load(f"{SAMPLE_DATA_DIR}/fun.tbl")
variables.load(f"{SAMPLE_DATA_DIR}/fun.var")

# ---- print the number of objects present in the above workspaces ----
len(comments), len(equations), len(identities), len(lists), len(scalars), len(tables), len(variables)

(317, 274, 48, 17, 161, 46, 394)

## Working with workspaces

To get the list of objects names present in a workspace, use the [names](../_generated/iode.Comments.names.rst#iode.Comments.names) attribute of the workspace. 
For example:

In [46]:
# get the list of all equations names
equations.names

['ACAF',
 'ACAG',
 'AOUC',
 'BENEF',
 'BQY',
 'BRUGP',
 'BVY',
 'CGU',
 'COEFON',
 'COTRES',
 'DEBT',
 'DPU',
 'DPUF',
 'DPUG',
 'DPUGO',
 'DPUH',
 'DPUU',
 'DTF',
 'DTH',
 'DTH1',
 'DTH1C',
 'EX',
 'EXC',
 'EXCC',
 'FLF',
 'FLG',
 'FLGR',
 'GAP',
 'GOSF',
 'GOSG',
 'GOSH',
 'GOSH_',
 'IDF',
 'IDG',
 'IDH',
 'IFU',
 'IHU',
 'IT',
 'ITCEE',
 'ITCR',
 'ITD',
 'ITEP',
 'ITF',
 'ITF5',
 'ITFC',
 'ITFGI',
 'ITFGO',
 'ITFQ',
 'ITGR',
 'ITI5R',
 'ITIFR',
 'ITIGR',
 'ITM',
 'ITMQ',
 'ITMQR',
 'ITNQ',
 'ITON',
 'ITONQ',
 'ITPL',
 'ITPR',
 'ITPS',
 'ITT',
 'IUG',
 'KL',
 'KLFHP',
 'KN5',
 'KNF',
 'KNFF',
 'KNFFY',
 'KNFY',
 'KNI',
 'KNIY',
 'NATY',
 'NFY',
 'NFYH',
 'OCUF',
 'OCUG',
 'OCUH',
 'PAF_',
 'PAG',
 'PAH',
 'PBBP',
 'PBNP',
 'PC',
 'PC_',
 'PDPUG',
 'PFI',
 'PFI_',
 'PFND',
 'PG',
 'PI5',
 'PIF',
 'PIG',
 'PKF',
 'PM',
 'PMAB',
 'PME',
 'PMS',
 'PMT',
 'POIL',
 'PQOG',
 'PROD',
 'PW3',
 'PWBG',
 'PWMAB',
 'PWMS',
 'PWXAB',
 'PWXS',
 'PX',
 'PXAB',
 'PXB',
 'PXE',
 'PXS',
 'PXT',
 'QAF'

To check if a name is present in a workspace, use the `in` operator. 
For example:

In [47]:
if 'ACAF' in equations:
    print("The 'ACAF' equation exists")
else:
    print("'ACAF' equation not found")

The 'ACAF' equation exists


To iterate over names of a workspace, simply use the Python syntax for the *for loop*:

In [48]:
print("Iterate over all equation names in the Equations workspace:")
for name in equations:
    if name.startswith("A"):
        print(name)

Iterate over all equation names in the Equations workspace:
ACAF
ACAG
AOUC


To get the current used sample for the Variables, use the [sample](../_generated/iode.Variables.sample.rst#iode.Variables.sample) attribute of the [variables](../_generated/iode.Variables.rst#iode.Variables) workspace:

In [49]:
# current used sample
variables.sample

'1960Y1:2015Y1'

To extract an object from a workspace, use the `[]` operator (see [here](../_generated/iode.Equations.__getitem__.rst) for examples):

In [50]:
# extract the equation named 'ACAF'
eq_ACAF = equations["ACAF"]

eq_ACAF

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+\nacaf4*(TIME=1995)',
         method = 'LSQ',
         from_period = '1980Y1',
         to_period = '1996Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

Then, you can modify the object and save it back to the workspace:

In [51]:
# extract 'ACAG' equation
eq_ACAG = equations["ACAG"]
print(f"old LEC of 'ACAG':    {eq_ACAG.lec}")
print(f"old SAMPLE of 'ACAG': {eq_ACAG.sample}")

# update the LEC expression and sample of the 'ACAG' equation
eq_ACAG.lec = "ACAG := ACAG[-1] + r VBBP[-1] + (0.006*VBBP[-1]*(TIME=2001) - 0.008*(TIME=2008))"
eq_ACAG.sample = "1990Y1:"

# save the modified 'ACAG' equation back to the Equations workspace
equations["ACAG"] = eq_ACAG
print(f"new LEC of 'ACAG':    {equations['ACAG'].lec}")
print(f"new SAMPLE of 'ACAG': {equations['ACAG'].sample}")

old LEC of 'ACAG':    ACAG := ACAG[-1]+r VBBP[-1]+(0.006*VBBP[-1]*(TIME=2001)-0.008*(TIME=2008))
old SAMPLE of 'ACAG': :
new LEC of 'ACAG':    ACAG := ACAG[-1] + r VBBP[-1] + (0.006*VBBP[-1]*(TIME=2001) - 0.008*(TIME=2008))
new SAMPLE of 'ACAG': 1990Y1:2015Y1


Another way to update an object is to use the syntax `database[name].attribute = value` directly (see [here](../_generated/iode.Equations.__setitem__.rst) for examples):

In [52]:
# Other option to update an object in a workspace: 
# use the syntax 'database[name].attribute = value' directly
equations["ACAF"].lec = "(ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)"
equations["ACAF"].method = EqMethod.MAX_LIKELIHOOD
equations["ACAF"].sample = "1990Y1:"

equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)',
         method = 'MAX_LIKELIHOOD',
         from_period = '1990Y1',
         to_period = '2015Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

The `[]` operator (see [here](../_generated/iode.Equations.__setitem__.rst)) can also be used to add an object:

In [53]:
# add a 'TEST' equation 
equations["TEST"] = "TEST := 0"

equations["TEST"]

Equation(endogenous = 'TEST',
         lec = 'TEST := 0',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

To delete an object from a workspace, use the `del` Python keyword (see [here](../_generated/iode.Equations.__delitem__.rst) for examples):

In [54]:
# remove the 'TEST' equation from the global equations workspace
del equations["TEST"]

# check that 'TEST' has been removed from the Equations workspace
"TEST" in equations

False

## Selecting periods from a Variable

It is possible to select the value(s) of a Variable for a given (range of) period(s) using the `[]` operator (see [here](../_generated/iode.Variables.__getitem__.rst)):

In [55]:
# get a whole variable
variables['ACAF']

[-2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 -2e+37,
 1.2130001,
 5.2020001,
 9.184,
 8.0790005,
 11.332,
 13.518001,
 15.784,
 16.544001,
 21.489,
 20.281,
 21.277,
 32.417999,
 24.446999,
 27.025002,
 24.504,
 27.560999,
 25.542,
 27.499001,
 25.353001,
 17.165001,
 23.771,
 26.240999,
 30.159,
 34.661999,
 8.1610022,
 -13.130997,
 32.171001,
 39.935799,
 29.645657,
 13.530404919696034,
 10.046610792200543,
 2.8679227364554634,
 -0.9292125090516451,
 -6.091564987568884,
 -14.582094462898064,
 -26.538789569788573,
 -28.987287982597536,
 -33.378425784295416,
 -38.40951778239742,
 -37.46350964127375,
 -37.82742883229439,
 -44.544792633543224,
 -55.55928981721873,
 -68.89465432262006,
 -83.34062511080091,
 -96.41041982848331]

In [56]:
# get the value of a variable for a given period
variables['ACAF', '2000Y1']

10.046610792200543

In [57]:
# get the values of a variable for a given periods range
variables['ACAF', '2000Y1:2010Y1']

[10.046610792200543,
 2.8679227364554634,
 -0.9292125090516451,
 -6.091564987568884,
 -14.582094462898064,
 -26.538789569788573,
 -28.987287982597536,
 -33.378425784295416,
 -38.40951778239742,
 -37.46350964127375,
 -37.82742883229439]

## Working with workspaces subsets

To extract a subset of a workspace, you can use either a string pattern or a list of names. 
The rules for the pattern are explained in the documentation of the `[]` operator (see [here](../_generated/iode.Variables.__getitem__.rst)). 

Examples of subsets:

In [58]:
# working with a subset of a the 'variables workspace' 
# 1. Using a string pattern
vars_subset = variables["A*;*_"]

vars_subset.names

['ACAF',
 'ACAG',
 'AOUC',
 'AOUC_',
 'AQC',
 'GAP_',
 'GOSH_',
 'PAF_',
 'PC_',
 'PFI_',
 'PROIHP_',
 'QAFF_',
 'QAF_',
 'QAI_',
 'QAT_',
 'QBBPPOT_',
 'QC_',
 'QQMAB_',
 'QS_',
 'TFPFHP_',
 'VAFF_',
 'VAF_',
 'VAI_',
 'VAT_',
 'VC_',
 'VS_',
 'WBF_',
 'WBU_',
 'WCF_',
 'WIND_',
 'WNF_',
 'YDH_',
 'ZZF_']

In [59]:
# working with a subset of a the 'variables workspace' 
# 2. Using a list of names
vars_subset = variables[["ACAF", "ACAG", "AQC", "BQY", "BVY"]]

vars_subset.names

['ACAF', 'ACAG', 'AQC', 'BQY', 'BVY']

Updating/adding/deleting an object from a subset will also affect the the global workspace:

In [60]:
# adding a 'DUMMY' variable
vars_subset["DUMMY"] = NA

# check that the 'DUMMY' variable has been added to the global 'variables' workspace
print(f"Is the new Variable 'DUMMY' in the global variables workspace?: {'DUMMY' in variables}")

variables["DUMMY", "1960Y1"]

Is the new Variable 'DUMMY' in the global variables workspace?: True


-2e+37

In [61]:
# updating the 'DUMMY' variable
vars_subset["DUMMY"] = 0

# check that the 'DUMMY' variable has been updated in the global 'variables' workspace
variables["DUMMY", "1960Y1"]

0.0

In [62]:
# deleting the 'DUMMY' variable
del vars_subset["DUMMY"]

# check that the 'DUMMY' variable has been deleted from the global 'variables' workspace
print(f"Is the new Variable 'DUMMY' in the global variables workspace?: {'DUMMY' in variables}")

Is the new Variable 'DUMMY' in the global variables workspace?: False


## Estimation

To estimate either one equation or a block of equations, use the [Equation.estimate](../_generated/iode.Equation.estimate.rst#iode.Equation.estimate) method of the equation object.

In [63]:
help(equations.estimate)

Help on method estimate in module iode.iode_python:

estimate(from_period: 'Union[str, Period]' = None, to_period: 'Union[str, Period]' = None, list_eqs: 'Union[str, List[str]]' = None) method of iode.iode_python.Equations instance
    estimate(self, from_period: Union[str, Period] = None, to_period: Union[str, Period] = None, list_eqs: Union[str, List[str]] = None)
    
    Estimate an equation or a block of equations.
    
    At the end of the estimation process, certain variables and scalars are automatically created 
    if the process has converged. These variables and scalars can be used for computational purposes and, 
    as they are part of the global workspace, can be saved for future use.
    
    The tests resulting from the last estimation are saved as scalars. The same applies to residuals, 
    left-hand and right-hand members of equations.
    
    Saved tests (as scalars) have the following names (`e<ith>_*` for the <ith> equation of the block):
    
        - `e<ith>

Example for one equation:

In [64]:
# ---- estimate coefficients of one equation ----
print(f"ACAF equation LEC: {equations['ACAF'].lec}")
print(f"ACAF equations coefficients (= scalars): {equations['ACAF'].coefficients}")
print(f"ACAF equations variables: {equations['ACAF'].variables}")

# reset scalars
for name in equations['ACAF'].coefficients:
    scalars[name] = 0., 1.

# estimate the 'ACAF' equation for the periods ranging from '1980Y1' to '1996Y1'
equations.estimate("1980Y1", "1996Y1", "ACAF")
# or equivalently
equations["ACAF"].estimate("1980Y1", "1996Y1")

print(f"Resulting values for the coefficient 'acaf1': {scalars['acaf1']}")
print(f"Resulting values for the coefficient 'acaf2': {scalars['acaf2']}")
print(f"Resulting values for the coefficient 'acaf4': {scalars['acaf4']}")

ACAF equation LEC: (ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)
ACAF equations coefficients (= scalars): ['acaf1', 'acaf2', 'acaf4']
ACAF equations variables: ['ACAF', 'VAF', 'GOSF', 'TIME']
Resulting values for the coefficient 'acaf1': Scalar(0.0157705, 1, 0.00124279)
Resulting values for the coefficient 'acaf2': Scalar(-7.96505e-06, 1, 1.34532e-06)
Resulting values for the coefficient 'acaf4': Scalar(-0.0085027, 1, 0.0018899)


Example for a block of equations:

In [65]:
# ---- estimate a block of equations ----
print(f"ACAF equation LEC: {equations['ACAF'].lec}")
print(f"ACAF equations coefficients (= scalars): {equations['ACAF'].coefficients}")
print(f"ACAF equations variables: {equations['ACAF'].variables}")
print(f"DPUH equation LEC: {equations['DPUH'].lec}")
print(f"DPUH equations coefficients (= scalars): {equations['DPUH'].coefficients}")
print(f"DPUH equations variables: {equations['DPUH'].variables}")

# reset scalars
for name in equations['ACAF'].coefficients:
    scalars[name] = 0., 1.
for name in equations['DPUH'].coefficients:
    scalars[name] = 0., 1.

# prepare equations (same block and method)
block = "ACAF;DPUH"
for name in block.split(";"):
    equations[name] = {"block": block, "method": "LSQ"}

# estimation the block 'ACAF;DPUH' for the periods ranging from '1980Y1' to '1996Y1'
equations.estimate("1980Y1", "1996Y1", block)

print(f"Resulting values for the coefficient 'acaf1': {scalars['acaf1']}")
print(f"Resulting values for the coefficient 'acaf2': {scalars['acaf2']}")
print(f"Resulting values for the coefficient 'acaf4': {scalars['acaf4']}")
print(f"Resulting values for the coefficient 'dpuh_1': {scalars['dpuh_1']}")
print(f"Resulting values for the coefficient 'dpuh_2': {scalars['dpuh_2']}")

ACAF equation LEC: (ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)
ACAF equations coefficients (= scalars): ['acaf1', 'acaf2', 'acaf4']
ACAF equations variables: ['ACAF', 'VAF', 'GOSF', 'TIME']
DPUH equation LEC: dln (DPUH/DPUHO):=dpuh_1+dpuh_2*dln(IHU/PI5)+dln PC
DPUH equations coefficients (= scalars): ['dpuh_1', 'dpuh_2']
DPUH equations variables: ['DPUH', 'DPUHO', 'IHU', 'PI5', 'PC']
Resulting values for the coefficient 'acaf1': Scalar(0.0157705, 1, 0.00136079)
Resulting values for the coefficient 'acaf2': Scalar(-7.96505e-06, 1, 1.47188e-06)
Resulting values for the coefficient 'acaf4': Scalar(-0.0085027, 1, 0.00206603)
Resulting values for the coefficient 'dpuh_1': Scalar(0.0109855, 1, 0.00481857)
Resulting values for the coefficient 'dpuh_2': Scalar(0.0574893, 1, 0.0368951)


## Simulation

To simulate a model, you must create and initialize an instance of the [Simulation](../_generated/iode.Simulation.rst#iode.Simulation) class:

In [66]:
from iode import Simulation

help(Simulation)

Help on class Simulation in module iode.iode_python:

class Simulation(builtins.object)
 |  Class for simulate models.
 |  
 |  Models
 |  ~~~~~~
 |  
 |  IN IODE, a model is simply a list of equations. No other construction is necessary: in this way, 
 |  to modify a model, the only thing to do is to modify the list that defines it. 
 |  For example, if a model is logically divided into 5 blocks, 5 lists of equations will be defined:: 
 |  
 |      BLOC1 : A, B, C, D
 |      BLOC2 : X, Y
 |      BLOC3 : C1, C2, C3, C4
 |      BLOC4 : X_A1, X_A2, X_A3
 |      BLOC5 : R1, R2, R3, S_12
 |      MODSIM : $BLOC1, $BLOC2, $BLOC3, $BLOC4, $BLOC5
 |  
 |  To simulate a model, all equations of the model must have been loaded or created. In addition, 
 |  all the variables and scalars used in the model equations must have been loaded or created. 
 |  Values of exogenous variables and of scalar cannot be :math:`NA` (*Not Available*) over the 
 |  simulation periods.
 |  
 |  Exchange Endo-Exo
 | 

In [67]:
from iode import SimulationSort

print(f"possible sort algorithms: {[member.name for member in SimulationSort]}")

possible sort algorithms: ['CONNEX', 'BOTH', 'NONE']


In [68]:
from iode import SimulationInitialization

print(f"possible initialization methods: {[member.name for member in SimulationInitialization]}")

possible initialization methods: ['TM1', 'TM1_A', 'EXTRA', 'EXTRA_A', 'ASIS', 'TM1_NA', 'EXTRA_NA']


In [69]:
# ---- simulation ----
print("Simulation() methods and properties:")
for name in dir(Simulation):
    if not name.startswith('_'):
        print(name)

Simulation() methods and properties:
convergence_threshold
debug
debug_newton
initialization_method
max_nb_iterations
max_nb_iterations_newton
model_calculate_SCC
model_compile
model_exchange
model_simulate
model_simulate_SCC
nb_passes
relax
sort_algorithm


In [70]:
# create and initialize a Simulation instance
simu = Simulation(sort_algorithm=SimulationSort.BOTH, initialization_method=SimulationInitialization.TM1)
print(f"Simulation convergence_threshold: {simu.convergence_threshold}")
print(f"Simulation relax: {simu.relax}")
print(f"Simulation max_nb_iterations: {simu.max_nb_iterations}")
print(f"Simulation sort_algorithm: {simu.sort_algorithm}")
print(f"Simulation initialization_method: {simu.initialization_method}")
print(f"Simulation debug: {simu.debug}")
print(f"Simulation nb_passes: {simu.nb_passes}")
print(f"Simulation debug_newton: {simu.debug_newton}")

Simulation convergence_threshold: 0.001
Simulation relax: 1.0
Simulation max_nb_iterations: 100
Simulation sort_algorithm: BOTH (Connex compon. + Triangulation)
Simulation initialization_method: TM1 (Y := Y[-1], if Y null or NA)
Simulation debug: False
Simulation nb_passes: 5
Simulation debug_newton: False


To run the simulation, call the [Simulation.model_simulate](../_generated/iode.Simulation.model_simulate.rst#iode.Simulation.model_simulate) method:

In [71]:
help(Simulation.model_simulate)

Help on cython_function_or_method in module iode.iode_python:

model_simulate(self, from_period: 'Union[str, Period]', to_period: 'Union[str, Period]', list_eqs: 'Union[str, List[str]]' = None)
    model_simulate(self, from_period: Union[str, Period], to_period: Union[str, Period], list_eqs: Union[str, List[str]] = None)
    
    Run the simulation of a model for a given sample.
    
    To simulate a model, all equations of the model must have been loaded or created. In addition, 
    all the variables and scalars used in the model equations must have been loaded or created. 
    Values of exogenous variables and of scalar cannot be :math:`NA` (*Not Available*) over the 
    simulation sample.
    
    Parameters
    ----------
    from_period : str or Period
        The starting period for the simulation.
    to_period : str or Period
        The ending period for the simulation.
    list_eqs : str or list(str), optional
        List of equations representing the model.
        Defau

In [72]:
print(f"exogenous variable 'UY': {equations['UY'].lec}")
print(f"endogenous variable 'XNATY': {identities['XNATY']}")
# reset values of exogenous variable
variables["UY", "2000Y1:2015Y1"] = 0.0

print(f"exogenous variable 'UY' before simulation:\n{variables['UY', '1998Y1:2005Y1']}")

# run the simulation for the periods range '2000Y1' to '2015Y1'
simu.model_simulate("2000Y1", "2015Y1")

print(f"exogenous variable 'UY' after simulation:\n{variables['UY', '1998Y1:2005Y1']}")

exogenous variable 'UY': UY := NATY-NDOMY-NIY-NGY-(EFXY-EFMY)-NFY
endogenous variable 'XNATY': grt NATY
exogenous variable 'UY' before simulation:
[671.46, 647.3906678388862, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
exogenous variable 'UY' after simulation:
[671.46, 647.3906678388862, 624.1781881798956, 645.0542977503601, 661.6074127102179, 676.5611515243486, 680.8992865590999, 682.9920328779863]


## Save IODE objects

To save the content of a workspace (or a subset of a workspace), use the [save()](../_generated/iode.Variables.save.rst#iode.Variables.save) method:

In [73]:
# ---- save workspace (or subset) ----
# save the whole workspace
equations.save('equations.eqs')

# save only a subset of the global variables workspace
vars_subset = variables[["ACAF", "ACAG", "AQC", "BQY", "BVY"]]
vars_subset.save('variables_subset.av')    

print("Check content of the variables_subset.av file:\n")
with open("variables_subset.av", "r") as f:
    print(f.read())
print()

Check content of the variables_subset.av file:

sample 1960Y1 2015Y1
ACAF na na na na na na na na na na 1.2130001 5.2020001 9.184 8.0790005 11.332 13.518001 15.784 16.544001 21.489 20.281 21.277 32.417999 24.446999 27.025002 24.504 27.560999 25.542 27.499001 25.353001 17.165001 23.771 26.240999 30.159 34.661999 8.1610022 -13.130997 32.171001 39.935799 29.645657 13.530404919696 10.0245731206742 2.59495261488497 -1.30484330804381 -6.43120286330679 -14.8973426392203 -26.8660050238949 -29.2393185662514 -33.5235533108231 -38.5307750051367 -37.2532299990484 -37.4445679355065 -44.1924169730606 -55.3005507377536 -68.8321197567019 -83.4352776524324 -96.5953921431648 
ACAG na na na na na na na na na na -11.028999 -15.847 -19.288002 -21.814999 -25.447002 -24.618999 -27.770998 -28.839001 -29.434998 -30.411001 -30.353001 -41.060997 -31.178001 -32.604 -30.237003 -38.061001 -31.939999 -35.59 -37.238003 -25.991001 -28.1721855713507 -30.934 -40.285999 -43.157997 -16.029003 -41.845993 -40.237 -32.93 -38

## Import/Export IODE workspaces to pandas Series and DataFrame

To import / export the content of the `comments`, `identities` and `lists` workspaces to a pandas Series object, use the [from_series()](../_generated/iode.Comments.from_series.rst#iode.Comments.from_series) and [to_series()](../_generated/iode.Comments.to_series.rst#iode.Comments.to_series) methods. 

Alternatively, you can use the [series](../_generated/iode.Comments.series.rst#iode.Comments.series) property to export the content of the `comments`, `identities` and `lists` workspaces to a pandas Series object.

In [74]:
import pandas as pd

# ---- to pandas Series ----
# See Comments/Identities/Lists.to_series and 
#     Comments/Identities/Lists.series

series_cmt = comments.to_series()
print(f"Comments as pandas Series:\n{series_cmt.info}")
print()

series_idt = identities.to_series()
print(f"Identities as pandas Series:\n{series_idt.info}")
print()

series_lst = lists.to_series()
print(f"Lists as pandas Series:\n{series_lst.info}")

# Alternatively

series_cmt = comments.series
series_idt = identities.series
series_lst = lists.series

# ---- from pandas Series ----
# See Comments/Identities/Lists.from_series

comments.from_series(series_cmt)
identities.from_series(series_idt)
lists.from_series(series_lst)

Comments as pandas Series:
<bound method Series.info of ACAF        Ondernemingen: ontvangen kapitaaloverdrachten.
ACAG     Totale overheid: netto ontvangen kapitaaloverd...
AOUC                             Kost per eenheid produkt.
AQC          Kost per eenheid produkt: kapitaal en arbeid.
BENEF             Ondernemingen: niet-uitgekeerde winsten.
                               ...                        
ZF       Indexeringscoëfficiënt voor de lonen in de pri...
ZJ       Indexeringscoëfficiënt voor de sociale uitkeri...
ZKF            Bezettingsgraad van de produktiecapaciteit.
ZX                          Saut d'index (correction en %)
ZZ_      Marktsector (ondernemingen en zelfstandigen): ...
Name: Comments, Length: 317, dtype: object>

Identities as pandas Series:
<bound method Series.info of AOUC      ((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+VAFF))...
AOUC_     exp(ln(((WCF/NFYH)/QL)+PKF/(QAFF/KNFFY))*(QAFF...
FLGR                                               FLG/VBBP
GAP2        

To import / export the content of the `equations`, `scalars` and `variables` workspaces to a pandas DataFrame object, use the [from_frame()](../_generated/iode.Variables.from_frame.rst#iode.Variables.from_frame) and [to_frame()](../_generated/iode.Variables.to_frame.rst#iode.Variables.to_frame) methods. 

Alternatively, you can use the [df](../_generated/iode.Variables.df.rst#iode.Variables.df) property to export the content of the `equations`, `scalars` and `variables` workspaces to a pandas DataFrame object.

In [75]:
# ---- to pandas DataFrame ----
# See Equations/Scalars/Variables.to_frame and
#     Equations/Scalars/Variables.df

df_eqs = equations.to_frame()
print(f"Equations as pandas DataFrame:\n{df_eqs.info}")
print()

df_scl = scalars.to_frame()
print(f"Scalars as pandas DataFrame:\n{df_scl.info}")
print()

df_vars = variables.to_frame()
print(f"Variables as pandas DataFrame:\n{df_vars.info}")

# Alternatively

df_eqs = equations.df
df_scl = scalars.df
df_vars = variables.df

# ---- from pandas DataFrame ----
# See Equations/Scalars/Variables.from_frame

equations.from_frame(df_eqs)
scalars.from_frame(df_scl)
variables.from_frame(df_vars)

Equations as pandas DataFrame:
<bound method DataFrame.info of                                                      lec method  \
ACAF   (ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + a...    LSQ   
ACAG   ACAG := ACAG[-1] + r VBBP[-1] + (0.006*VBBP[-1...    LSQ   
AOUC   AOUC:=((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+...    LSQ   
BENEF  d BENEF :=d(VBNP-(IT+ITCEE)+SUB-DPUU-(WCF+SSFF...    LSQ   
BQY                                    BQY:=(YK+YN)/PBBP    LSQ   
...                                                  ...    ...   
YSSF                                  dln YSSF:=dln WBF_    LSQ   
YSSG                       YSSG := SSF+SSH-(YSSF+COTRES)    LSQ   
ZF                       grt ZF :=grt PC+ZX-0.05*grt PME    LSQ   
ZJ                      grt ZJ :=grt PC +ZX-0.05*grt PME    LSQ   
ZZF_                                    ZZF_ := ZZF_[-1]    LSQ   

              sample comment instruments      block  corr    stdev     meany  \
ACAF   1980Y1:1996Y1                      ACAF;DPUH 

## Import/Export the Variables workspace to LArray

To import / export the content of the `variables` workspaces to a LArray Array object, use the [from_array()](../_generated/iode.Variables.from_array.rst#iode.Variables.from_array) and [to_array()](../_generated/iode.Variables.to_array.rst#iode.Variables.to_array) methods:

In [76]:
import larray as la

# ---- to LArray Array ----
# See Variables.to_array

arr_vars = variables.to_array()
print(f"Variables as LArray Array:\n{arr_vars.info}")

# ---- from LArray array ----
# See Variables.from_array

variables.from_array(arr_vars)

Variables as LArray Array:
400 x 56
 names [400]: 'ACAF' 'ACAG' 'AOUC' ... '_YOBS1' '_YRES0' '_YRES1'
 time [56]: '1960Y1' '1961Y1' '1962Y1' ... '2013Y1' '2014Y1' '2015Y1'
dtype: float64
memory used: 175.00 Kb


## Execute IODE report commands/files

To run an IODE command, use the [execute_command()](../_generated/iode.execute_command.rst#iode.execute_command) function:

In [77]:
from iode import execute_command

# ---- execute IODE commands ----
print("Execute IODE commands (useful for IODE functions and commands not yet ported to Python):")
execute_command("$WsClearVar")
execute_command("$WsSample 2000Y1 2005Y1")
execute_command("$DataCalcVar A t+1")
execute_command("$DataCalcVar B t-1")
execute_command("$DataCalcVar C A/B")
execute_command("$DataCalcVar D grt A")
execute_command("$WsSaveVar test_var.av")
with open("test_var.av", "r") as f:
    print(f.read())

Execute IODE commands (useful for IODE functions and commands not yet ported to Python):
sample 2000Y1 2005Y1
A 1 2 3 4 5 6 
B -1 0 1 2 3 4 
C -1 na 3 2 1.66666666666667 1.5 
D na 100 50 33.3333333333333 25 20 



To run an entire IODE report (i.e. file with the '.rep' extension), call the [execute_report()](../_generated/iode.execute_report.rst#iode.execute_report) function:

In [78]:
from iode import execute_report

# ---- execute IODE reports ----
print("Execute an IODE report -> execute_report()")
with open("create_var.rep", "w") as f:
    f.write("$WsClearVar\n")
    f.write("$WsSample 2000Y1 2005Y1\n")
    f.write("$DataCalcVar %1% t+1 \n")
    f.write("$DataCalcVar %2% t-1 \n")
    f.write("$DataCalcVar %3% %1%/%2%\n")
    f.write("$DataCalcVar %4% grt %1% \n")
    f.write("$WsSaveVar test_var.av\n")

execute_report("create_var.rep", ["A", "B", "C", "D"])

with open("test_var.av", "r") as f:
    print(f.read())


Execute an IODE report -> execute_report()
sample 2000Y1 2005Y1
A 1 2 3 4 5 6 
B -1 0 1 2 3 4 
C -1 na 3 2 1.66666666666667 1.5 
D na 100 50 33.3333333333333 25 20 

